# Trend Detection

In [13]:
import numpy as np
import pandas as pd
import seaborn as sns
import time
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
import requests
import pandas_ta as ta

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120

import warnings
warnings.filterwarnings('ignore')

In [14]:
entrade_headers = {
        'authority': 'services.entrade.com.vn',
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'en-US,en;q=0.9',
        'dnt': '1',
        'origin': 'https://banggia.dnse.com.vn',
        'referer': 'https://banggia.dnse.com.vn/',
        'sec-ch-ua': '"Edge";v="114", "Chromium";v="114", "Not=A?Brand";v="24"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'cross-site',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 Edg/114.0.1788.0'
    }
DNSE_DATA_HISTORY_URL = 'https://services.entrade.com.vn/chart-api/v2/ohlcs/derivative'

In [15]:
def getStockHistoryData(ticker, timestamp_from=0, timestamp_to=0):
    if timestamp_from == 0:
        three_months = date.today() + relativedelta(months=-3)
        timestamp_from = datetime.strptime(three_months.strftime("%m/%d/%Y") + ', 00:00:0', "%m/%d/%Y, %H:%M:%S")\
            .timestamp()
    if timestamp_to == 0:
        timestamp_to = datetime.strptime(date.today().strftime("%m/%d/%Y") + ', 23:59:00', "%m/%d/%Y, %H:%M:%S")\
            .timestamp()

    params = {
        "resolution": "5",
        "symbol": "VN30F1M",
        "from": int(timestamp_from),
        "to": int(timestamp_to)
    }

    x = requests.get(DNSE_DATA_HISTORY_URL, params=params, headers=entrade_headers)
    response = x.json()

    import numpy as np
    import pandas as pd

    timestamp = np.array(response['t']).astype(int)
    close = np.array(response['c']).astype(float)
    open = np.array(response['o']).astype(float)
    high = np.array(response['h']).astype(float)
    low = np.array(response['l']).astype(float)
    volume = np.array(response['v']).astype(int)

    dataset = pd.DataFrame({'Time': timestamp, 'Open': list(open), 'High': list(high), 'Low': list(low),
                            'Close': list(close), 'Volume': list(volume)},
                           columns=['Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
    return dataset
def prepareData(htd):
    if 'Time' in htd.columns:
        from datetime import datetime

        htd['DateStr'] = htd.apply(
            lambda x: datetime.fromtimestamp(x['Time']).strftime("%Y-%m-%d %H:%M:%S"), axis=1)

    htd['Date'] = pd.to_datetime(htd['DateStr'])
    # htd['Date'] = htd['Date'] + pd.DateOffset(hours=7)
    ticker_data = htd.set_index('Date')
    ticker_data.drop(columns=['Time', 'DateStr'], inplace=True)
    return ticker_data

## Load price data from DNSE

In [16]:
ticker = "VN30F1M"
htd = getStockHistoryData(ticker, 0, 0)
ticker_data = prepareData(htd)
ticker_data = ticker_data.dropna()
ticker_data

,Open,High,Low,Close,Volume
Date,,,,,
2024-04-19 09:00:00,1201.4,1201.4,1196.3,1197.9,13049
2024-04-19 09:05:00,1197.9,1199.2,1197.3,1198.2,4446
2024-04-19 09:10:00,1197.8,1198.7,1196.8,1198.0,3872
2024-04-19 09:15:00,1197.9,1198.5,1195.8,1198.0,7339
2024-04-19 09:20:00,1198.1,1200.2,1198.0,1198.8,7059
...,...,...,...,...,...
2024-07-19 14:15:00,1301.2,1301.6,1298.3,1299.5,9058
2024-07-19 14:20:00,1299.1,1301.8,1295.3,1301.6,12762
2024-07-19 14:25:00,1302.4,1302.6,1300.1,1302.2,8096


In [17]:
data2 = ticker_data.copy()
data2["ema_line"] = ta.ema(data2["Close"], length=20)
data2['above_ma'] = data2.apply(lambda r: 1 if r['Close'] > r['ema_line'] else 0, axis=1)
data2['below_ma'] = data2.apply(lambda r: 1 if r['Close'] < r['ema_line'] else 0, axis=1)
data2['total_above_ma'] = data2['above_ma'].rolling(150).sum()
data2['total_below_ma'] = data2['below_ma'].rolling(150).sum()
data2['total_above_ma_1w'] = data2['above_ma'].rolling(250).sum()
data2['total_below_ma_1w'] = data2['below_ma'].rolling(250).sum()
data2['trend_3d'] = data2.apply(lambda r: 'switch' if r['total_above_ma'] == r['total_below_ma'] else ('up' if r['total_above_ma'] > r['total_below_ma'] else 'down'), axis=1)
data2['trend_1w'] = data2.apply(lambda r: 'switch' if r['total_above_ma_1w'] == r['total_below_ma_1w'] else ('up' if r['total_above_ma_1w'] > r['total_below_ma_1w'] else 'down'), axis=1)

In [18]:
data2

,Open,High,Low,Close,Volume,ema_line,above_ma,below_ma,total_above_ma,total_below_ma,total_above_ma_1w,total_below_ma_1w,trend_3d,trend_1w
Date,,,,,,,,,,,,,,
2024-04-19 09:00:00,1201.4,1201.4,1196.3,1197.9,13049,NaN,0,0,NaN,NaN,NaN,NaN,down,down
2024-04-19 09:05:00,1197.9,1199.2,1197.3,1198.2,4446,NaN,0,0,NaN,NaN,NaN,NaN,down,down
2024-04-19 09:10:00,1197.8,1198.7,1196.8,1198.0,3872,NaN,0,0,NaN,NaN,NaN,NaN,down,down
2024-04-19 09:15:00,1197.9,1198.5,1195.8,1198.0,7339,NaN,0,0,NaN,NaN,NaN,NaN,down,down
2024-04-19 09:20:00,1198.1,1200.2,1198.0,1198.8,7059,NaN,0,0,NaN,NaN,NaN,NaN,down,down
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-19 14:15:00,1301.2,1301.6,1298.3,1299.5,9058,1301.748096,0,1,60.0,90.0,116.0,134.0,down,down
2024-07-19 14:20:00,1299.1,1301.8,1295.3,1301.6,12762,1301.733992,0,1,60.0,90.0,115.0,135.0,down,down
2024-07-19 14:25:00,1302.4,1302.6,1300.1,1302.2,8096,1301.778373,1,0,60.0,90.0,115.0,135.0,down,down


In [19]:
today = data2[data2.index > '2024-07-19 08:00:00']
today

,Open,High,Low,Close,Volume,ema_line,above_ma,below_ma,total_above_ma,total_below_ma,total_above_ma_1w,total_below_ma_1w,trend_3d,trend_1w
Date,,,,,,,,,,,,,,
2024-07-19 09:00:00,1304.6,1305.3,1303.3,1304.4,6590,1296.532951,1,0,69.0,81.0,107.0,143.0,down,down
2024-07-19 09:05:00,1304.4,1305.4,1304.3,1304.4,2285,1297.282194,1,0,69.0,81.0,107.0,143.0,down,down
2024-07-19 09:10:00,1304.4,1304.6,1303.9,1304.0,2085,1297.921985,1,0,69.0,81.0,107.0,143.0,down,down
2024-07-19 09:15:00,1304.0,1304.7,1303.2,1303.4,3109,1298.443701,1,0,69.0,81.0,107.0,143.0,down,down
2024-07-19 09:20:00,1303.5,1303.5,1302.3,1303.2,4233,1298.896682,1,0,69.0,81.0,107.0,143.0,down,down
2024-07-19 09:25:00,1303.2,1309.1,1303.2,1309.0,8842,1299.858902,1,0,69.0,81.0,107.0,143.0,down,down
2024-07-19 09:30:00,1309.2,1311.2,1308.0,1308.4,8595,1300.672340,1,0,69.0,81.0,107.0,143.0,down,down
2024-07-19 09:35:00,1308.3,1309.8,1308.1,1308.7,3564,1301.436879,1,0,69.0,81.0,107.0,143.0,down,down
2024-07-19 09:40:00,1308.7,1309.2,1308.3,1309.0,1881,1302.157176,1,0,69.0,81.0,108.0,142.0,down,down
